# Example Notebook

In this notebook we show how to use the library to implement computionally-efficient DP-SGD with JAX.

# 0. Setup (skip until 1. if you don't need the details)

In [1]:
import argparse
import os
import math
import time
import warnings
import jax

import numpy as np

## 0.1 Enviroment variables

In [2]:
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".90"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"

jax.clear_caches()

## 0.2 Use GPU or CPU?

In [3]:
USE_GPU = False

## 0.3 Arguments

Here you can change the value of the arguments by changing the default values.

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument("--lr", default=0.001, type=float, help="learning rate")
parser.add_argument("--num_steps", default=10, type=int, help="Number of steps")
parser.add_argument("--logical_bs", default=100, type=int, help="Logical batch size")
parser.add_argument("--clipping_norm", default=1, type=float, help="max grad norm")

parser.add_argument("--target_epsilon", default=8, type=float, help="target epsilon")
parser.add_argument("--target_delta", default=1e-5, type=float, help="target delta")

parser.add_argument("--physical_bs", default=2, type=int, help="Physical Batch Size")
parser.add_argument("--accountant", default="pld", type=str, help="The privacy accountant for DP training.")

parser.add_argument("--seed", default=1234, type=int)
args = parser.parse_args(args=[])


print("Used args:", args, flush=True)


Used args: Namespace(lr=0.001, num_steps=10, logical_bs=100, clipping_norm=1, target_epsilon=8, target_delta=1e-05, physical_bs=2, accountant='pld', seed=1234)


# 1. Setting up dataset, model and DP accounting
We show you how to setup the dataset, model and how the DP accounting works.

## 1.1 Dataset

We load the dataset from [Hugging Face](https://huggingface.co/) but the only important thing is to have the data available as arrays. Hugging Face supports this nicely but there might be other or even better ways to achieve this.

In [ ]:
from dpsgdjax.data import load_from_huggingface

train_images, train_labels, test_images, test_labels = load_from_huggingface(
    "uoft-cs/cifar10", cache_dir=None, feature_name="img"
)
ORIG_IMAGE_DIMENSION, RESIZED_IMAGE_DIMENSION = 32, 32
train_images = train_images[train_labels < 2].transpose(0, 3, 1, 2)
train_labels = train_labels[train_labels < 2]
test_images = test_images[test_labels < 2].transpose(0, 3, 1, 2)
test_labels = test_labels[test_labels < 2]


num_classes = 2
dataset_size = len(train_labels)

train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

## 1.2 Model

We create a `flax.training.train_state.TrainState` and load pre-trained weights from [Hugging Face](https://huggingface.co/) using the `create_train_state` function that we provide in `src.models.py`. In this particular example, we load the weights of a simple Conv_Net.

In [ ]:
from dpsgdjax.models import create_train_state
from collections import namedtuple

optimizer_config = namedtuple("Config", ["learning_rate"])
optimizer_config.learning_rate = args.lr

state = create_train_state(
    model_name="small",
    num_classes=num_classes,
    image_dimension=RESIZED_IMAGE_DIMENSION,
    optimizer_config=optimizer_config,
)

load model name small


## 1.3 DP accounting

First, we compute the `subsampling_ratio` based on the `dataset_size` and the (expected) `logical_bs`. Then we compute the required DP-SGD `noise_std` based on the `subsampling_ratio` and the `num_steps` for a particular pair of `target_epsilon` and `target_delta` using a privacy accountant. At the moment the Privacy Loss Distributions (PLDs) and RDP accounting from the google [dp_accounting](https://github.com/google/differential-privacy/tree/main/python/dp_accounting) library are supported. 

*Note: You can also use the accounting tooling of other libraries such as the PyTorch based [opacus](https://github.com/pytorch/opacus).*

In [ ]:
from dpsgdjax.dp_accounting_utils import calculate_noise
if dataset_size * args.target_delta > 1.0:
    warnings.warn("Your delta might be too high.")

subsampling_ratio = 1 / math.ceil(dataset_size / args.logical_bs)

noise_std = calculate_noise(
        sample_rate=subsampling_ratio,
        target_epsilon=args.target_epsilon,
        target_delta=args.target_delta,
        steps=args.num_steps,
        accountant=args.accountant,
)

# 2. Function to process one physical batch

First we define the function that computes per example gradients (`compute_per_example_gradients_physical_batch`) and clips them (`clip_and_accumulate_physical_batch`). This function can be jit compiled and then used in the full training loop later (see 3.).

In [ ]:
from dpsgdjax.jax_mask_efficient import (
    compute_per_example_gradients_physical_batch,
    add_trees,
    clip_physical_batch,
    accumulate_physical_batch,
)

@jax.jit
def process_physical_batch(t, params):
    (
        state,
        accumulated_clipped_grads,
        logical_batch_X,
        logical_batch_y,
        masks,
    ) = params
    # slice
    start_idx = t * args.physical_bs
    pb = jax.lax.dynamic_slice(
        logical_batch_X,
        (start_idx, 0, 0, 0, 0),
        (args.physical_bs, 1, 3, ORIG_IMAGE_DIMENSION, ORIG_IMAGE_DIMENSION),
    )
    yb = jax.lax.dynamic_slice(logical_batch_y, (start_idx,), (args.physical_bs,))
    mask = jax.lax.dynamic_slice(masks, (start_idx,), (args.physical_bs,))

    # compute grads and clip
    per_example_gradients = compute_per_example_gradients_physical_batch(state, pb, yb, num_classes)
    clipped_grads_from_pb = clip_physical_batch(per_example_gradients, args.clipping_norm)
    sum_of_clipped_grads_from_pb = accumulate_physical_batch(clipped_grads_from_pb, mask)
    accumulated_clipped_grads = add_trees(accumulated_clipped_grads, sum_of_clipped_grads_from_pb)

    return (
        state,
        accumulated_clipped_grads,
        logical_batch_X,
        logical_batch_y,
        masks,
    )

# 3. Full training loop

The below cell executes the main training loop. It consists of the following parts at every step:

- Poission sampling of the logical batch size (`poisson_sample_logical_batch_size`): This gives us the logical batch size using Poisson subsampling.
- Rounding up of the logical batch size so that there are full physical batches (`setup_physical_batches`): This rounds up the logical batch size so that it is divisible in full physical batches
- Padding of the logical batches (`get_padded_logical_batch`): Here we load the actual images and labels.
- Computation of the per sample gradients (`jax.lax.fori_loop` using the previously defined `process_physical_batch`): This efficiently computes the per-example gradients of the logical batch.
- Addition of noise (`add_Gaussian_noise`): Add the required noise to the accumulated gradients of the logical batch. 
- Update of the model (`update_model`): Apply the gradient update to the model weights.

At the end of a step the following things are executed:
- Computation of the throughput: Compute the number of processed examples divided by the time spent.
- Evaluate the model (`model_evaluation`): Compute the test accuracy.
- Compute the spent privacy budget (`compute_epsilon`): Compute the spent privacy budget using a privacy accountant.

In [ ]:
from dpsgdjax.dp_accounting_utils import compute_epsilon
from dpsgdjax.jax_mask_efficient import (
    get_padded_logical_batch,
    model_evaluation,
    add_Gaussian_noise,
    poisson_sample_logical_batch_size,
    setup_physical_batches,
    update_model,
)

times = []
logical_batch_sizes = []

for t in range(args.num_steps):
    sampling_rng = jax.random.key(t + 1)
    batch_rng, binomial_rng, noise_rng = jax.random.split(sampling_rng, 3)

    #######
    # poisson subsample
    actual_batch_size = poisson_sample_logical_batch_size(
        binomial_rng=binomial_rng, dataset_size=dataset_size, q=subsampling_ratio
    )

    # determine padded_logical_bs so that there are full physical batches
    # and create appropriate masks to mask out unnessary elements later
    masks, n_physical_batches = setup_physical_batches(
        actual_logical_batch_size=actual_batch_size,
        physical_bs=args.physical_bs,
    )

    # get random padded logical batches that are slighly larger actual batch size
    padded_logical_batch_X, padded_logical_batch_y = get_padded_logical_batch(
        batch_rng=batch_rng,
        padded_logical_batch_size=len(masks),
        train_X=train_images,
        train_y=train_labels,
    )

    padded_logical_batch_X = padded_logical_batch_X.reshape(-1, 1, 3, ORIG_IMAGE_DIMENSION, ORIG_IMAGE_DIMENSION)

    # cast to GPU
    if USE_GPU:
        padded_logical_batch_X = jax.device_put(padded_logical_batch_X, jax.devices("gpu")[0])
        padded_logical_batch_y = jax.device_put(padded_logical_batch_y, jax.devices("gpu")[0])
        masks = jax.device_put(masks, jax.devices("gpu")[0])

    print("##### Starting gradient accumulation #####", flush=True)
    ### gradient accumulation
    params = state.params

    accumulated_clipped_grads0 = jax.tree.map(lambda x: 0.0 * x, params)

    start = time.time()

    # Main loop
    _, accumulated_clipped_grads, *_ = jax.lax.fori_loop(
        0,
        n_physical_batches,
        process_physical_batch,
        (
            state,
            accumulated_clipped_grads0,
            padded_logical_batch_X,
            padded_logical_batch_y,
            masks,
        ),
    )
    noisy_grad = add_Gaussian_noise(noise_rng, accumulated_clipped_grads, noise_std, args.clipping_norm)

    # update
    state = jax.block_until_ready(update_model(state, noisy_grad))

    end = time.time()
    duration = end - start

    times.append(duration)
    logical_batch_sizes.append(actual_batch_size)

    print(f"throughput at iteration {t}: {actual_batch_size / duration}", flush=True)

    acc_iter = model_evaluation(
        state, test_images, test_labels, batch_size=10, orig_image_dimension=ORIG_IMAGE_DIMENSION, use_gpu=USE_GPU
    )
    print(f"accuracy at iteration {t}: {acc_iter}", flush=True)

    # Compute privacy guarantees
    epsilon, delta = compute_epsilon(
        noise_multiplier=noise_std,
        sample_rate=subsampling_ratio,
        steps=t + 1,
        target_delta=args.target_delta,
        accountant=args.accountant,
    )
    privacy_results = {"accountant": args.accountant, "epsilon": epsilon, "delta": delta}
    print(privacy_results, flush=True)

##### Starting gradient accumulation #####
throughput at iteration 0: 88.8964614868164
accuracy at iteration 0: 0.453000009059906
{'accountant': 'pld', 'epsilon': 5.95716589775049, 'delta': 9.999999999657339e-06}
##### Starting gradient accumulation #####
throughput at iteration 1: 270.6031188964844
accuracy at iteration 1: 0.5394999980926514
{'accountant': 'pld', 'epsilon': 6.508521466962656, 'delta': 9.999999999519879e-06}
##### Starting gradient accumulation #####
throughput at iteration 2: 299.2416076660156
accuracy at iteration 2: 0.6230000257492065
{'accountant': 'pld', 'epsilon': 6.837246686024637, 'delta': 9.999999999487661e-06}
##### Starting gradient accumulation #####
throughput at iteration 3: 259.6731262207031
accuracy at iteration 3: 0.6610000133514404
{'accountant': 'pld', 'epsilon': 7.080491327747894, 'delta': 9.99999999946578e-06}
##### Starting gradient accumulation #####
throughput at iteration 4: 632.8670043945312
accuracy at iteration 4: 0.5525000095367432
{'accoun

## 3.1 Final Model evaluation
Here we computate of the throughput (num processed examples/time spent) and final test accuracy (`model_evaluation`).

In [11]:
acc_last = model_evaluation(state, test_images, test_labels, batch_size=10, use_gpu=USE_GPU, orig_image_dimension=ORIG_IMAGE_DIMENSION)

print("times \n", times, flush=True)

print("batch sizes \n ", logical_batch_sizes, flush=True)

print("accuracy at end of training", acc_last, flush=True)
thr = np.mean(np.array(logical_batch_sizes) / np.array(times))
print("throughput", thr)

times 
 [1.023662805557251, 0.384326696395874, 0.31412744522094727, 0.39280152320861816, 0.1595911979675293, 0.3310582637786865, 0.2965869903564453, 0.3153975009918213, 0.16562175750732422, 0.32752299308776855]
batch sizes 
  [Array(91., dtype=float32), Array(104., dtype=float32), Array(94., dtype=float32), Array(102., dtype=float32), Array(101., dtype=float32), Array(100., dtype=float32), Array(92., dtype=float32), Array(97., dtype=float32), Array(104., dtype=float32), Array(108., dtype=float32)]
accuracy at end of training 0.717
throughput 342.87716612836016
